In [2]:
%matplotlib inline
from os.path import basename, join
from skimage.io import imread
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler
#from keras.callbacks import ReduceLROnPlateau
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Conv2D
#from keras.layers.pooling import MaxPooling2D
#from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, History
from keras.models import load_model
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from os.path import basename, join
from glob import glob
from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D

from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.utils import print_summary
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop


def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res

def train_classifier(train_gt, train_img_dir, fast_train = True):    
    new_height = 200
    new_width = 200
    num_classes = 50
    
    jpeg_list = sorted(glob(join(train_img_dir, '*jpg')))
    num_samples = len(jpeg_list)
    
    X = np.zeros([num_samples, new_height, new_width, 3])
    y = np.zeros([num_samples, num_classes])
    yy = np.zeros([num_samples])
    
    count = 0
    for path in jpeg_list:
        image = imread(path)
        
        # resize the image
        scale_width = image.shape[1] / new_width
        scale_height = image.shape[0] / new_height
        image_resized = resize(image, (new_height, new_width))
        
        # handle grayscale images
        if len(image_resized.shape) == 2:
            tmp_image = np.zeros([new_height, new_width, 3])
            tmp_image[:, :, 0] = image_resized
            tmp_image[:, :, 1] = image_resized
            tmp_image[:, :, 2] = image_resized
            image_resized = tmp_image
        
        X[count] = image_resized
        class_ind = int(train_gt[basename(path)])
        y[count, class_ind] = 1
        yy[count] = int(class_ind)
        
        count = count + 1
    
    X_train = X
    yy_train = y
    X_test = None
    yy_test = None
    
    if fast_train is not True:
        X_train, X_test, yy_train, yy_test = train_test_split(X, yy, test_size = 0.2,
                                                            random_state = 2017, shuffle = True,
                                                            stratify = yy)
        
    y_train = np.zeros([len(yy_train), num_classes])
    y_test = np.zeros([len(yy_test), num_classes])
    
    for i in range(len(yy_train)):
        class_ind = int(yy_train[i]) 
        y_train[i, class_ind] = 1
        
    for i in range(len(yy_test)):
        class_ind = int(yy_test[i])
        y_test[i, class_ind] = 1
    
    base_model = ResNet50(include_top = False, weights='imagenet', input_shape = [new_height, new_width, 3])
    #print_summary(base_model)
    
    x = base_model.output
    x = Flatten()(x)
    
    #x = Dropout(0.2)(x)
    #x = Dense(2 * num_classes, activation = 'relu')(x)
    
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation = 'softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    num_layers = len(base_model.layers)
    ind_layer = 1
    for layer in base_model.layers:
        if ind_layer < (num_layers - 7): # do not froze the last five layers
            layer.trainable = False
        ind_layer = ind_layer + 1
    
    #print_summary(model)
    #return

    # compile the model (should be done *after* setting layers to non-trainable)
    rmps_prop = RMSprop(lr=0.00001)
    
    model.compile(optimizer = rmps_prop, loss = 'categorical_crossentropy', metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath = 'birds_model.hdf5', verbose = 1, save_best_only = True)

    epochs = 4000
    batch_size = 50

    if fast_train is True:
        epochs = 1
    
    datagen = ImageDataGenerator(
        featurewise_center = False,
        featurewise_std_normalization = False,
        rotation_range = 20,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True)
    
    #model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
    #                steps_per_epoch=len(x_train) / 32, epochs=epochs)

    #model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size),
    #                    validation_data = (X_test, y_test),
    #                    steps_per_epoch = len(X_train) / batch_size,
    #                    epochs=epochs,
    #                    callbacks = [checkpointer])

    # train the model on the new data
    #print_summary(model)
    model.fit(X_train, y_train,
              batch_size=batch_size,
              validation_data = (X_test, y_test),
              epochs=epochs,
              callbacks = [checkpointer]
             )

    #model.save("birds_model.hdf5")
    
    pass


def classify(model, test_img_dir):
    new_height = 200
    new_width = 200
    num_classes = 50
    
    jpeg_list = sorted(glob(join(test_img_dir, '*jpg')))
    num_samples = len(jpeg_list)
    
    X = np.zeros([num_samples, new_height, new_width, 3])
    y = np.zeros([num_samples, num_classes])
    
    count = 0
    test_gt = {}
    test_filenames = []
    for path in jpeg_list:
        image = imread(path)
        test_gt[basename(path)] = 0
        
        # resize the image
        scale_width = image.shape[1] / new_width
        scale_height = image.shape[0] / new_height
        image_resized = resize(image, (new_height, new_width))
        
        # handle grayscale images
        if len(image_resized.shape) == 2:
            tmp_image = np.zeros([new_height, new_width, 3])
            tmp_image[:, :, 0] = image_resized
            tmp_image[:, :, 1] = image_resized
            tmp_image[:, :, 2] = image_resized
            image_resized = tmp_image
        
        X[count] = image_resized
        test_filenames.append(basename(path))
        count = count + 1
    
    yy = model.predict(X)
    y = np.where(yy == 1)[1]
    
    for picture_ind in range(len(test_filenames)):
        test_filename = test_filenames[picture_ind]
        test_gt[test_filename] = y[picture_ind]

    return test_gt


# call to train a model
train_dir = 'data/00_input/train'
train_img_dir = 'data/00_input/train/images'
train_gt = read_csv(join(train_dir, 'gt.csv'))
train_classifier(train_gt, train_img_dir, False)

# call to test trained model
#test_img_dir = 'data/00_input/test/images'
#model = load_model('facepoints_model.hdf5')
#detected_points = detect(model, test_img_dir)



c:\program files\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train on 2000 samples, validate on 500 samples
Epoch 1/4000
2000/2000 [==============================] - 105s 52ms/step - loss: 4.4122 - acc: 0.0220 - val_loss: 4.1207 - val_acc: 0.0200
Epoch 2/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 4.1735 - acc: 0.0355 - val_loss: 4.1774 - val_acc: 0.0200
Epoch 3/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 4.0438 - acc: 0.0450 - val_loss: 4.5348 - val_acc: 0.0200
Epoch 4/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 3.8533 - acc: 0.0575 - val_loss: 5.0892 - val_acc: 0.0180
Epoch 5/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 3.7223 - acc: 0.0810 - val_loss: 5.8669 - val_acc: 0.0200
Epoch 6/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 3.5781 - acc: 0.1060 - val_loss: 6.6257 - val_acc: 0.0140
Epoch 7/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 3.4321 - acc: 0.1345 - val_loss: 7.3342 - va

2000/2000 [==============================] - 20s 10ms/step - loss: 1.3383 - acc: 0.7045 - val_loss: 1.8330 - val_acc: 0.5400
Epoch 31/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.3194 - acc: 0.7090 - val_loss: 1.8006 - val_acc: 0.5280
Epoch 32/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.2537 - acc: 0.7295 - val_loss: 1.7698 - val_acc: 0.5280
Epoch 33/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.2146 - acc: 0.7395 - val_loss: 1.7432 - val_acc: 0.5360
Epoch 34/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.1824 - acc: 0.7500 - val_loss: 1.7159 - val_acc: 0.5460
Epoch 35/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.1246 - acc: 0.7705 - val_loss: 1.6927 - val_acc: 0.5480
Epoch 36/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 1.0876 - acc: 0.7715 - val_loss: 1.6702 - val_acc: 0.5560
Epoch 37/4000
2000/2000 [================

2000/2000 [==============================] - 20s 10ms/step - loss: 0.5221 - acc: 0.9265 - val_loss: 1.3527 - val_acc: 0.6280
Epoch 58/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.5065 - acc: 0.9305 - val_loss: 1.3419 - val_acc: 0.6280
Epoch 59/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4791 - acc: 0.9350 - val_loss: 1.3334 - val_acc: 0.6340
Epoch 60/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4628 - acc: 0.9495 - val_loss: 1.3222 - val_acc: 0.6320
Epoch 61/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4535 - acc: 0.9510 - val_loss: 1.3130 - val_acc: 0.6380
Epoch 62/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4446 - acc: 0.9465 - val_loss: 1.3050 - val_acc: 0.6400
Epoch 63/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4115 - acc: 0.9505 - val_loss: 1.2965 - val_acc: 0.6400
Epoch 64/4000
2000/2000 [================

2000/2000 [==============================] - 20s 10ms/step - loss: 0.1839 - acc: 0.9915 - val_loss: 1.1835 - val_acc: 0.6600
Epoch 84/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1684 - acc: 0.9940 - val_loss: 1.1802 - val_acc: 0.6600
Epoch 85/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1714 - acc: 0.9930 - val_loss: 1.1782 - val_acc: 0.6600
Epoch 86/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1590 - acc: 0.9955 - val_loss: 1.1744 - val_acc: 0.6580
Epoch 87/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1544 - acc: 0.9955 - val_loss: 1.1719 - val_acc: 0.6580
Epoch 88/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1410 - acc: 0.9985 - val_loss: 1.1679 - val_acc: 0.6660
Epoch 89/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1454 - acc: 0.9950 - val_loss: 1.1655 - val_acc: 0.6660
Epoch 90/4000
2000/2000 [================

2000/2000 [==============================] - 20s 10ms/step - loss: 0.0523 - acc: 1.0000 - val_loss: 1.1116 - val_acc: 0.6740
Epoch 111/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.0519 - acc: 0.9995 - val_loss: 1.1107 - val_acc: 0.6720
Epoch 112/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.0471 - acc: 0.9995 - val_loss: 1.1089 - val_acc: 0.6760
Epoch 113/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0464 - acc: 1.0000 - val_loss: 1.1091 - val_acc: 0.6760
Epoch 114/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0431 - acc: 1.0000 - val_loss: 1.1094 - val_acc: 0.6760
Epoch 115/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.0403 - acc: 1.0000 - val_loss: 1.1089 - val_acc: 0.6740
Epoch 116/4000
2000/2000 [==============================] - 20s 10ms/step - loss: 0.0376 - acc: 1.0000 - val_loss: 1.1087 - val_acc: 0.6720
Epoch 117/4000
2000/2000 [=========

Epoch 138/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0138 - acc: 1.0000 - val_loss: 1.0901 - val_acc: 0.6860
Epoch 139/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0122 - acc: 1.0000 - val_loss: 1.0924 - val_acc: 0.6880
Epoch 140/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0118 - acc: 1.0000 - val_loss: 1.0919 - val_acc: 0.6840
Epoch 141/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0117 - acc: 1.0000 - val_loss: 1.0922 - val_acc: 0.6820
Epoch 142/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0109 - acc: 1.0000 - val_loss: 1.0903 - val_acc: 0.6800
Epoch 143/4000
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0105 - acc: 1.0000 - val_loss: 1.0919 - val_acc: 0.6820
Epoch 144/4000
 500/2000 [======>.......................] - ETA: 11s - loss: 0.0115 - acc: 1.00

KeyboardInterrupt: 